# Key word extraction algorithm for claims

In [1]:
import os, json
from helper_function import count_valid_posts, get_claims, get_premises
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [2]:
with open('../data/v3_70_filled/Should-I-invest-in-Bitcoin_with_labels_v3_70_filled.json', 'r') as f:
    file_content = json.load(f)
print('Get valid posts: ', count_valid_posts(file_content))
claims = get_claims(file_content)
premises = get_premises(file_content)

Get valid posts:  (41, 70)


## Keyword extraction algorithms

In [3]:
from keyword_extractor import TfIdfExtractor

if_idf = TfIdfExtractor()

In [4]:
if_idf.extract_keywords(premises)

[[('closed', 0.4273649338385547),
  ('issue', 0.4273649338385547),
  ('everything', 0.21368246691927734)],
 [('understand', 0.5200573223867475),
  ('trouble', 0.2840450554716571),
  ('quickly', 0.2840450554716571)],
 [('phrase', 0.3013803684116225),
  ('research', 0.3013803684116225),
  ('focused', 0.3013803684116225)],
 [('short', 0.41704271879918237),
  ('term', 0.41704271879918237),
  ('longer', 0.24375391486447495)],
 [('sense', 0.4551581970210135),
  ('invest', 0.3681895435604688),
  ('jumped', 0.22757909851050676)],
 [('market', 0.281478547072952),
  ('earn', 0.23084333648011293),
  ('studying', 0.23084333648011293)],
 [('gold', 0.36862850755739845),
  ('digital', 0.35682843752904436),
  ('like', 0.35682843752904436)],
 [('gold', 0.5414356960381959),
  ('long', 0.28886276089144275),
  ('less', 0.21657427841527835)],
 [('verified', 0.3439024655172588),
  ('addition', 0.3439024655172588),
  ('predictable', 0.3439024655172588)],
 [('created', 0.6078539739833883),
  ('bitcoins', 0.41

## KeyBERT Extractor

In [5]:
from keyword_extractor import KeyBERTExtractor

In [6]:
keybert_extractor = KeyBERTExtractor()

In [7]:
keybert_extractor.extract_keywords(premises)

[[('bitcoin', 0.4979), ('investing', 0.4563), ('opportunities', 0.2908)],
 [('bitcoin', 0.5205), ('concept', 0.3693), ('scam', 0.3509)],
 [('buying', 0.3241), ('late', 0.3223), ('worried', 0.2866)],
 [('investments', 0.4398), ('bitcoin', 0.4299), ('short', 0.3415)],
 [('bitcoin', 0.4461), ('cryptocurrency', 0.4451), ('invest', 0.4367)],
 [('cryptocurrency', 0.5786), ('bitcoin', 0.5045), ('investing', 0.4918)],
 [('bitcoin', 0.679), ('digital', 0.3529), ('securely', 0.3377)],
 [('inflation', 0.5708), ('currency', 0.4643), ('monetary', 0.4407)],
 [('bitcoin', 0.6092), ('bitcoins', 0.5908), ('monetary', 0.4478)],
 [('bitcoins', 0.4029), ('transaction', 0.2113), ('new', 0.1906)],
 [('bitcoin', 0.4549), ('bitcoins', 0.4311), ('bank', 0.3732)],
 [('bitcoin', 0.5736), ('bitcoins', 0.5541), ('monetary', 0.5055)],
 [('bitcoin', 0.5141), ('monetary', 0.3766), ('payments', 0.3209)],
 [('bitcoin', 0.4754), ('currency', 0.4743), ('finance', 0.3899)],
 [('bitcoin', 0.5566), ('volatility', 0.455), ('